# Recognizing Flatlanders

## Data preprocessing

In [1]:
import h5py
from keras import backend as K
from sklearn.model_selection import train_test_split
import numpy as np

Using TensorFlow backend.


Read the training and test data from the HDF5 files.

In [2]:
with h5py.File('train_data.h5', 'r') as h5_file:
    x_train = np.array(h5_file['x_values'])
    y_train = np.array(h5_file['y_values'])
with h5py.File('test_data.h5', 'r') as h5_file:
    x_test = np.array(h5_file['x_values'])
    y_test = np.array(h5_file['y_values'])

Determine whether the backend expects the channel to come first or last in the input shape.

In [3]:
img_channels, img_rows, img_cols = 1, x_train.shape[1], x_train.shape[2]

if K.image_data_format() == 'channels_first':
    shape_ord = (img_channels, img_rows, img_cols)
else:  # channel_last
    shape_ord = (img_rows, img_cols, img_channels)

Reshape the data accordingly.

In [4]:
x_train = x_train.reshape((x_train.shape[0], ) + shape_ord)
x_test = x_test.reshape((x_test.shape[0], ) + shape_ord)

Convert the input data to single precision floating point, and scale between 0 and 1.

In [5]:
x_train = x_train.astype(np.float32)/255.0
x_test = x_test.astype(np.float32)/255.0

Verify input data format and shape.

In [6]:
print(x_train.shape, x_train.dtype, x_train.min(), x_train.max())
print(x_test.shape, x_test.dtype, x_test.min(), x_test.max())

(60000, 100, 100, 1) float32 0.0 1.0
(20000, 100, 100, 1) float32 0.0 1.0


Verify output data format and shape.

In [7]:
print(y_train.shape, y_train.dtype, y_train.min(), y_train.max())
print(y_test.shape, y_test.dtype, y_test.min(), y_test.max())

(60000, 3) uint8 0 1
(20000, 3) uint8 0 1


Seed the random number generator to ensure reproducible results in this notebook.

In [ ]:
np.random.seed(1234)

In [8]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train)

In [9]:
print(x_train.shape, x_train.dtype, x_train.min(), x_train.max())
print(x_val.shape, x_val.dtype, x_val.min(), x_val.max())

(45000, 100, 100, 1) float32 0.0 1.0
(15000, 100, 100, 1) float32 0.0 1.0


In [10]:
print(y_train.shape, y_train.dtype, y_train.min(), y_train.max())
print(y_val.shape, y_val.dtype, y_val.min(), y_val.max())

(45000, 3) uint8 0 1
(15000, 3) uint8 0 1


## Model definition

Import modules requred for CNN defintion.

In [11]:
from keras.layers import Activation, Conv2D, Dense, Dropout, Flatten, LeakyReLU
from keras.models import Sequential
from keras.optimizers import Adam
from history_utils import plot_history

Define the CNN's architecture.

In [17]:
nr_filters = 32
nr_classes = y_train.shape[-1]
conv_x_size, conv_y_size = 5, 5
model = Sequential()
model.add(Conv2D(nr_filters, (conv_x_size, conv_y_size), strides=2,
                 padding='valid', input_shape=shape_ord))
model.add(LeakyReLU(0.1))
model.add(Dropout(0.4))
model.add(Conv2D(nr_filters*2, (conv_x_size, conv_y_size), strides=2,
                 padding='valid'))
model.add(LeakyReLU(0.1))
model.add(Dropout(0.4))
model.add(Conv2D(nr_filters*4, (conv_x_size, conv_y_size), strides=2,
                 padding='valid'))
model.add(LeakyReLU(0.1))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(nr_classes))
model.add(Activation('softmax'))

In [18]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 48, 48, 32)        832       
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 48, 48, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 22, 22, 64)        51264     
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 22, 22, 64)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 22, 22, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 9, 9, 128)         204928    
__________

Compile the CNN, specifying the loss function.

In [19]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(),
              metrics=['accuracy'])

Train the network.

In [20]:
history = model.fit(x_train, y_train, epochs=100, batch_size=64,
                    validation_data=(x_val, y_val))

Train on 45000 samples, validate on 15000 samples
Epoch 1/100
45000/45000 [==============================] - 8s 181us/step - loss: 0.8608 - acc: 0.5504 - val_loss: 0.5470 - val_acc: 0.7399
Epoch 2/100
45000/45000 [==============================] - 8s 171us/step - loss: 0.4403 - acc: 0.8055 - val_loss: 0.1920 - val_acc: 0.9422
Epoch 3/100
45000/45000 [==============================] - 8s 171us/step - loss: 0.2768 - acc: 0.8881 - val_loss: 0.1290 - val_acc: 0.9642
Epoch 4/100
45000/45000 [==============================] - 8s 171us/step - loss: 0.2162 - acc: 0.9157 - val_loss: 0.1581 - val_acc: 0.9364
Epoch 5/100
45000/45000 [==============================] - 8s 171us/step - loss: 0.1764 - acc: 0.9318 - val_loss: 0.0868 - val_acc: 0.9630
Epoch 6/100
45000/45000 [==============================] - 8s 171us/step - loss: 0.1588 - acc: 0.9385 - val_loss: 0.0722 - val_acc: 0.9728
Epoch 7/100
45000/45000 [==============================] - 8s 171us/step - loss: 0.1413 - acc: 0.9448 - val_loss: 0.

Save the trained CNN to an HDF5 file so that it can be reused.

In [ ]:
model.save('identify_individuals.h5')

Show the evolution of the loss and accuracy during the trainnig.

In [ ]:
plot_history(history)

Evaluate the CNN's performance on the test data set.

In [21]:
model.evaluate(x_test, y_test)

20000/20000 [==============================] - 1s 65us/step


[0.04177384389367512, 0.9902]